In [1]:
import torch

print(torch.__version__)

torch.cuda.empty_cache()

2.6.0+cu124


In [2]:
from numba import cuda
device = cuda.get_current_device(); device.reset()

In [3]:
!nvidia-smi

Wed Apr  9 12:33:25 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:37:00.0 Off |                    0 |
| N/A   49C    P0              38W / 400W |     25MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [4]:
import os

# GPU 0만 사용하도록 제한 (다른 GPU는 보이지 않게 됨)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
import torch

# 현재 Setup 되어있는 device 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())
print(torch.cuda.get_device_name(device))

Available devices  1
Current cuda device  0
NVIDIA A100-SXM4-40GB


In [6]:
gpu_stats = torch.cuda.get_device_properties(0)

gpu_stats

_CudaDeviceProperties(name='NVIDIA A100-SXM4-40GB', major=8, minor=0, total_memory=40339MB, multi_processor_count=108, uuid=c75b47fc-231a-03c4-1b9d-84fd5a9c2142, L2_cache_size=40MB)

In [7]:

# CUDA 장치의 주요 버전과 부 버전을 가져옵니다.
major_version, minor_version = torch.cuda.get_device_capability()
major_version, minor_version

(8, 0)

In [10]:
%%capture
import os

# [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
!pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

In [12]:
%%capture
import os
!pip install --no-deps unsloth vllm
# [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
# Skip restarting message in Colab
import sys, re, requests; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

# vLLM requirements - vLLM breaks Colab due to reinstalling numpy
f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
with open("vllm_requirements.txt", "wb") as file:
    file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
!pip install -r vllm_requirements.txt

In [ ]:
# 훈련을 위한 세팅을 여기서 한번에 결정합시다.

from huggingface_hub import login, create_repo

hf_token = os.getenv("HUGGINGFACE_TOKEN")

# 1. basemodel 
base_model = "Bllossom/llama-3.2-Korean-Bllossom-3B" # "unsloth/Qwen2.5-0.5B", "unsloth/Qwen2.5-1.5B", "unsloth/Qwen2.5-3B"

# 2. dataset repo 
dataset_repo = "HueyWoo/edie8_pinklab_dataset" # "MarkrAI/KoCommercial-Dataset" / beomi/KoAlpaca-v1.1a

# 3. save model repo 
model_repo = "HueyWoo/edie8_pinklab_llama3.2_ko_3B"
try:
    create_repo(model_repo)
except:
    print("Repository already exists")

# 4. save model name 
save_model_name = "edie8_pinklab_llama3.2_ko_3B"


# 5. FastLanguageModel.get_peft_model 옵션 
## 5.1 r: 저차원 행렬의 **랭크(rank)**를 나타냅니다. 이 값은 모델의 미세 조정(fine-tuning) 과정에서 학습되는 가중치 업데이트 행렬의 차원을 결정. , r 값이 클수록 학습되는 파라미터의 수가 증가하여 모델의 표현력이 높아질 수 있지만, 그만큼 메모리 사용량과 계산 비용도 증가
rank_r = 8  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
## 5.2 스케일링 팩터로, LoRA 모듈이 생성한 가중치 업데이트의 크기를 조절. 모델 학습 시 가중치 조정의 강도를 결정하며, 일반적으로 r 값의 2배로 설정하는 것을 권장
lora_alpha_scale = 8
### 5.3 드롭아웃
lora_dropout_rate=0.0
random_state = 3407

# 6. SFTTrainer 옵션 세팅 
batch_size = 8
dataset_num_proc=8  # 데이터 처리에 사용할 프로세스 수
num_train_epochs=1  # 5
max_steps=1000       # 500
optim="adamw_8bit"             # 최적화 알고리즘
lr_scheduler_type="linear"      # cosine / "linear"  # 학습률 스케줄러 유형
weight_decay = 0.01
random_seed = 123
do_eval = True

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.



In [25]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = base_model, # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.0.dev0. vLLM: 0.8.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Bllossom/llama-3.2-Korean-Bllossom-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [26]:
model = FastLanguageModel.get_peft_model(
    model,
    r=rank_r,  # 0보다 큰 어떤 숫자도 선택 가능! 8, 16, 32, 64, 128이 권장됩니다.
    lora_alpha=lora_alpha_scale,  # LoRA 알파 값을 설정합니다.
    lora_dropout=lora_dropout_rate,  # 드롭아웃을 지원합니다.
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # 타겟 모듈을 지정합니다.
    bias="none",  # 바이어스를 지원합니다.
    # True 또는 "unsloth"를 사용하여 매우 긴 컨텍스트에 대해 VRAM을 30% 덜 사용하고, 2배 더 큰 배치 크기를 지원합니다.
    use_gradient_checkpointing="unsloth",
    random_state=random_state,  # 난수 상태를 설정합니다.
    use_rslora=False,  # 순위 안정화 LoRA를 지원합니다.
    loftq_config=None,  # LoftQ를 지원합니다.
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [29]:
from datasets import load_dataset


# dataset = load_dataset(dataset_repo, split="train").select(range(1000))
dataset = load_dataset(dataset_repo, split="train")

dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 420
})

In [30]:
# AlpacaPrompt를 사용하여 지시사항을 포맷팅하는 함수입니다.
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""



In [31]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

# 주어진 예시들을 포맷팅하는 함수입니다.
def formatting_prompts_func(examples):
    instructions = examples["instruction"]  # 지시사항을 가져옵니다.
    outputs = examples["output"]  # 출력값을 가져옵니다.
    texts = []  # 포맷팅된 텍스트를 저장할 리스트입니다.
    for instruction, output in zip(instructions, outputs):
        # EOS_TOKEN을 추가해야 합니다. 그렇지 않으면 생성이 무한히 진행될 수 있습니다.
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,  # 포맷팅된 텍스트를 반환합니다.
    }


In [32]:
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

In [33]:
dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 420
})

In [34]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset,
        eval_dataset=dataset,
        dataset_text_field = "text",
        max_seq_length = max_seq_length,
        dataset_num_proc = dataset_num_proc,
        packing = False, # Can make training 5x faster for short sequences.
        args = TrainingArguments(
            per_device_train_batch_size = batch_size,
            gradient_accumulation_steps = 4,
            warmup_steps = 5,
            # num_train_epochs = 1, # Set this for 1 full training run.
            max_steps = max_steps,
            learning_rate = 2e-4,
            fp16 = not is_bfloat16_supported(),
            bf16 = is_bfloat16_supported(),
            logging_steps = 1,
            optim = "adamw_8bit",
            weight_decay = weight_decay,
            lr_scheduler_type = lr_scheduler_type,
            seed = random_seed,
            output_dir = "outputs",
            report_to = "none", # Use this for WandB etc
        ),
    )

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/420 [00:00<?, ? examples/s]

In [35]:
# 현재 메모리 상태를 보여주는 코드
gpu_stats = torch.cuda.get_device_properties(0)  # GPU 속성 가져오기
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 시작 시 예약된 GPU 메모리 계산
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPU의 최대 메모리 계산
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU 이름과 최대 메모리 출력
print(f"{start_gpu_memory} GB of memory reserved.")  # 예약된 메모리 양 출력

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.394 GB.
10.279 GB of memory reserved.


In [36]:
trainer_stats = trainer.train()  # 모델을 훈련시키고 통계를 반환합니다.

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 420 | Num Epochs = 77 | Total steps = 1,000
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 12,156,928/3,000,000,000 (0.41% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.501300
2,3.479900
3,3.466100
4,3.402300
5,3.407800
6,3.142000
7,2.946200
8,2.626300
9,2.414300
10,2.216900


In [37]:
# 최종 메모리 및 시간 통계를 보여줍니다.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # 사용된 최대 메모리를 GB 단위로 계산합니다.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRA를 위해 사용된 메모리를 GB 단위로 계산합니다.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # 최대 메모리 대비 사용된 메모리의 비율을 계산합니다.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # 최대 메모리 대비 LoRA를 위해 사용된 메모리의 비율을 계산합니다.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # 훈련에 사용된 시간을 초 단위로 출력합니다.
print(
    # 훈련에 사용된 시간을 분 단위로 출력합니다.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # 훈련을 위해 예약된 최대 메모리를 GB 단위로 출력합니다.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # 최대 메모리 대비 예약된 메모리의 비율을 출력합니다.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # 최대 메모리 대비 훈련을 위해 예약된 메모리의 비율을 출력합니다.

1330.2582 seconds used for training.
22.17 minutes used for training.
Peak reserved memory = 10.279 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 26.093 %.
Peak reserved memory for training % of max memory = 0.0 %.


In [38]:
from transformers import StoppingCriteria, StoppingCriteriaList


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id

    def __call__(self, input_ids, scores, **kwargs):
        # 리스트로 변환하여 정지 토큰이 존재하는지 확인합니다.
        return self.stop_token_id in input_ids[0].tolist()



# end_token을 설정
stop_token = tokenizer.eos_token
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[0]

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

In [39]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)

In [40]:
# dataset에서 무작위로 하나의 예제 선택
import random
random_index = random.randint(0, len(dataset) - 1)
print("random_index : ",random_index)
example = dataset[random_index]

# 입력 준비
instruction = example['instruction']
print(instruction)


random_index :  26
앞으로 조금 더 빠르게 가


In [41]:

# FastLanguageModel을 이용하여 추론 속도를 2배 빠르게 설정합니다.
FastLanguageModel.for_inference(model)

# 토큰화
inputs = tokenizer(
    [
        alpaca_prompt.format(
            example['instruction'],  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


# 생성
outputs = model.generate(
    **inputs,
    max_new_tokens=max_seq_length,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

# 디코딩 및 출력
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("생성된 응답:")
print(generated_text)

# 실제 응답 출력 (비교를 위해)
print("\n실제 정답:")
print(example['output'])

생성된 응답:
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
앞으로 조금 더 빠르게 가

### Response:
[ANSWER]: '주인님의 요청에 따라 더 빠르게 앞으로 가겠습니다.' [EMOTION]: 'HOPEFUL' [ACTION]: 'FORWARD' [TIME]: '2'

실제 정답:
[ANSWER]: '주인님의 요청에 따라 더 빠르게 앞으로 가겠습니다.' [EMOTION]: 'HOPEFUL' [ACTION]: 'FORWARD' [TIME]: '2'


In [42]:
# dataset에서 무작위로 하나의 예제 선택
random_index = random.randint(0, len(dataset) - 1)
print("random_index : ",random_index)
example = dataset[random_index]

# 입력 준비
instruction = example['instruction']
print(instruction)
# 토큰화
inputs = tokenizer(
    [
        alpaca_prompt.format(
            example['instruction'],  # 지시사항
            "",  # 출력 - 생성을 위해 이 부분을 비워둡니다!
        )
    ],
    return_tensors="pt",
).to("cuda")


_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=max_seq_length,  # 최대 생성 토큰 수를 설정합니다.
    stopping_criteria=stopping_criteria  # 생성을 멈출 기준을 설정합니다.
)

# 실제 응답 출력 (비교를 위해)
print("\n실제 정답:")
print(example['output'])

random_index :  137
왼쪽으로 2m 더 나아가 줘
<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
왼쪽으로 2m 더 나아가 줘

### Response:
[ANSWER]: '알겠습니다, 주인님. 왼쪽으로 이동하겠습니다.' [EMOTION]: 'LOVE' [ACTION]: 'TURN LEFT' [TIME]: '2'<|eot_id|>

실제 정답:
[ANSWER]: '알겠습니다, 주인님. 왼쪽으로 이동하겠습니다.' [EMOTION]: 'LOVE' [ACTION]: 'TURN LEFT' [TIME]: '2'


In [ ]:
# 저는 다 저장하려고 합니다. 
quantization_methods = ["f16", "q8_0", "q5_k_m", "q4_k_m"]
# quantization_methods = ["q4_k_m", "q5_k_m"]

In [ ]:
# # Local에 저장 
# model.save_pretrained_gguf(
#     huggingface_repo + "-gguf",
#     tokenizer=tokenizer,
#     quantization_method=quantization_method,
# )

In [49]:
# Hub 에 GGUF 업로드
model.push_to_hub_gguf(
    model_repo + "-gguf",
    tokenizer,
    quantization_method=quantization_methods,
    token=hf_token,
)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 196.86 out of 251.54 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:00<00:00, 115.12it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16', 'q8_0', 'q4_k_m', 'q5_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at HueyWoo/edie8_pinklab_llama3.2_ko_3B-gguf into f16 GGUF format.
The output location will be /home/llm_khw/Repo/fine_tuning/HueyWoo/edie8_pinklab_llama3.2_ko_3B-gguf/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: edie8_pinklab_llama3.2_ko_3B-gguf
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading mode

unsloth.F16.gguf:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/HueyWoo/edie8_pinklab_llama3.2_ko_3B-gguf
Unsloth: Uploading GGUF to Huggingface Hub...


...

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/HueyWoo/edie8_pinklab_llama3.2_ko_3B-gguf
Unsloth: Uploading GGUF to Huggingface Hub...


unsloth.Q5_K_M.gguf:   0%|          | 0.00/2.32G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Saved GGUF to https://huggingface.co/HueyWoo/edie8_pinklab_llama3.2_ko_3B-gguf
